In [49]:
# Import the relevant libraries.
import AbstractMCMC
#using AbstractMCMC
using Distributions
using Random

In [50]:
# Define a sampler type.
struct MetropolisHastings{T, D} <: AbstractMCMC.AbstractSampler 
    init_θ::T
    proposal::D
end

# Default constructors.
MetropolisHastings(init_θ::Real) = MetropolisHastings(init_θ, Normal(0,1))
MetropolisHastings(init_θ::Vector{<:Real}) = MetropolisHastings(init_θ, MvNormal(length(init_θ),1))

MetropolisHastings

In [141]:
rand(MvNormal(2,1))

2-element Array{Float64,1}:
  0.9073340617753408
 -0.856018701587421

In [92]:
rks = MetropolisHastings([0,1])

MetropolisHastings{Array{Int64,1},MvNormal{Float64,PDMats.ScalMat{Float64},FillArrays.Zeros{Float64,1,Tuple{Base.OneTo{Int64}}}}}([0, 1], ZeroMeanIsoNormal(
dim: 2
μ: 2-element Zeros{Float64}
Σ: [1.0 0.0; 0.0 1.0]
)
)

In [85]:
MvNormal(2,1)

ZeroMeanIsoNormal(
dim: 2
μ: 2-element Zeros{Float64}
Σ: [1.0 0.0; 0.0 1.0]
)


In [86]:
Normal(0,1)

Normal{Float64}(μ=0.0, σ=1.0)

In [17]:
rst = MetropolisHastings(0.5)
tr = MetropolisHastings(0.5, 0.2)

MetropolisHastings{Float64,Float64}(0.5, 0.2)

In [150]:
struct state 
    θ::Any
end

In [51]:
# Define a model type. Stores the log density function.
struct DensityModel{F<:Function} <: AbstractMCMC.AbstractModel
    ℓπ::F
end

In [52]:
# Create a very basic Transition type, only stores the 
# parameter draws and the log probability of the draw.
struct Transition{T, L}
    θ::T
    lp::L
end

# Store the new draw and its log density.
Transition(model::DensityModel, θ) = Transition(θ, ℓπ(model, θ))

Transition

In [95]:
test = Transition(model, [0,1]).θ

2-element Array{Int64,1}:
 0
 1

In [71]:
insupport(θ) = θ[2] >= 0
dist(θ) = Normal(θ[1], θ[2])
density(θ) = insupport(θ) ? sum(logpdf.(dist(θ), data)) : -Inf

density (generic function with 1 method)

In [75]:
A = 8

8

In [76]:
A::Int

8

In [110]:
typeof(Normal(0,1))

Normal{Float64}

In [115]:
sig = [1 0
        0 1]

2×2 Array{Int64,2}:
 1  0
 0  1

In [118]:

tst_dist = MvNormal([0,0],sig)

FullNormal(
dim: 2
μ: [0.0, 0.0]
Σ: [1.0 0.0; 0.0 1.0]
)


In [139]:
rand(tst_dist, 4)

2×4 Array{Float64,2}:
 -0.652451  0.392309  -0.489743   0.0893783
  1.46704   1.15042    0.636674  -0.167119

In [73]:
model = DensityModel(density)

DensityModel{typeof(density)}(density)

In [91]:
typeof([1,2,3,4])

Array{Int64,1}

In [90]:
model.ℓπ([0,1])

-654.4133956379703

In [151]:
# Define the first step! function, which is called at the 
# beginning of sampling. Return the initial parameter used
# to define the sampler.
function AbstractMCMC.step(
    rng::AbstractRNG,
    model::DensityModel,
    spl::MetropolisHastings,
    N::Integer,
    ::Nothing;
    kwargs...
)
    return Transition(model, spl.init_θ)
end

In [152]:

function AbstractMCMC.step(
    rng::AbstractRNG,
    model::DensityModel,
    spl::MetropolisHastings,
    N::Integer,
    ::Nothing;
    kwargs...
)
    return  spl.init_θ
end

In [108]:
s = rand(MvNormal(0,1),3)
s

0×3 Array{Float64,2}

In [54]:
# Define a function that makes a basic proposal depending on a univariate
# parameterization or a multivariate parameterization.
propose(spl::MetropolisHastings, model::DensityModel, θ::Real) = 
    Transition(model, θ + rand(spl.proposal))
propose(spl::MetropolisHastings, model::DensityModel, θ::Vector{<:Real}) = 
    Transition(model, θ + rand(spl.proposal))
propose(spl::MetropolisHastings, model::DensityModel, t::Transition) =
    propose(spl, model, t.θ)

# Calculates the probability `q(θ|θcond)`, using the proposal distribution `spl.proposal`.
q(spl::MetropolisHastings, θ::Real, θcond::Real) = logpdf(spl.proposal, θ - θcond)
q(spl::MetropolisHastings, θ::Vector{<:Real}, θcond::Vector{<:Real}) =
    logpdf(spl.proposal, θ - θcond)
q(spl::MetropolisHastings, t1::Transition, t2::Transition) = q(spl, t1.θ, t2.θ)

# Calculate the density of the model given some parameterization.
ℓπ(model::DensityModel, θ) = model.ℓπ(θ)
ℓπ(model::DensityModel, t::Transition) = t.lp

# Define the other step function. Returns a Transition containing
# either a new proposal (if accepted) or the previous proposal 
# (if not accepted).
function AbstractMCMC.step(
    rng::AbstractRNG,
    model::DensityModel,
    spl::MetropolisHastings,
    ::Integer,
    θ_prev::Transition;
    kwargs...
)
    # Generate a new proposal.
    θ = propose(spl, model, θ_prev)

    # Calculate the log acceptance probability.
    α = ℓπ(model, θ) - ℓπ(model, θ_prev) + q(spl, θ_prev, θ) - q(spl, θ, θ_prev)

    # Decide whether to return the previous θ or the new one.
    if log(rand(rng)) < min(α, 0.0)
        return θ
    else
        return θ_prev
    end
end

In [145]:
using Random
rng

LoadError: [91mUndefVarError: rng not defined[39m

In [153]:
function AbstractMCMC.step(
    rng::AbstractRNG,
    model::DensityModel,
    spl::MetropolisHastings,
    θ_prev::state;
    kwargs...
)
    # Generate a new proposal.
    θ = propose(spl, model, θ_prev)

    # Calculate the log acceptance probability.
    α = ℓπ(model, θ) - ℓπ(model, θ_prev) + q(spl, θ_prev, θ) - q(spl, θ, θ_prev)

    # Decide whether to return the previous θ or the new one.
    if log(rand(rng)) < min(α, 0.0)
        return θ.θ
    else
        return θ_prev.θ
    end
end

In [55]:
# A basic chains constructor that works with the Transition struct we defined.
function AbstractMCMC.bundle_samples(
    rng::AbstractRNG, 
    ℓ::DensityModel, 
    s::MetropolisHastings, 
    N::Integer, 
    ts::Vector{<:Transition},
    chain_type::Type{Any};
    param_names=missing,
    kwargs...
)
    # Turn all the transitions into a vector-of-vectors.
    vals = copy(reduce(hcat,[vcat(t.θ, t.lp) for t in ts])')

    # Check if we received any parameter names.
    if ismissing(param_names)
        param_names = ["Parameter $i" for i in 1:(length(first(vals))-1)]
    end

    # Add the log density field to the parameter names.
    push!(param_names, "lp")

    # Bundle everything up and return a Chains struct.
    return Chains(vals, param_names, (internals=["lp"],))
end

In [155]:
typeof(spl)

MetropolisHastings{Array{Float64,1},MvNormal{Float64,PDMats.ScalMat{Float64},FillArrays.Zeros{Float64,1,Tuple{Base.OneTo{Int64}}}}}

In [154]:
# Generate a set of data from the posterior we want to estimate.
data = rand(Normal(5, 3), 30)

# Define the components of a basic model.
insupport(θ) = θ[2] >= 0
dist(θ) = Normal(θ[1], θ[2])
density(θ) = insupport(θ) ? sum(logpdf.(dist(θ), data)) : -Inf

# Construct a DensityModel.
model = DensityModel(density)

# Set up our sampler with initial parameters.
spl = MetropolisHastings([0.0, 0.0])

# Sample from the posterior.
chain = sample(model, spl, 100000; param_names=["μ", "σ"])

LoadError: [91mMethodError: no method matching step(::Random._GLOBAL_RNG, ::DensityModel{typeof(density)}, ::MetropolisHastings{Array{Float64,1},MvNormal{Float64,PDMats.ScalMat{Float64},FillArrays.Zeros{Float64,1,Tuple{Base.OneTo{Int64}}}}}; param_names=["μ", "σ"])[39m
[91m[0mClosest candidates are:[39m
[91m[0m  step(::AbstractRNG, ::DensityModel, ::MetropolisHastings, [91m::Integer[39m, [91m::Transition[39m; kwargs...) at In[54]:23[39m
[91m[0m  step(::AbstractRNG, ::DensityModel, ::MetropolisHastings, [91m::Transition[39m; kwargs...) at In[58]:1[39m
[91m[0m  step(::AbstractRNG, ::DensityModel, ::MetropolisHastings, [91m::Integer[39m, [91m::Nothing[39m; kwargs...) at In[152]:2[39m
[91m[0m  ...[39m

In [57]:
data

30-element Array{Float64,1}:
  4.437528231492961
  2.2001126959384374
  6.787010926118304
 10.277018877593662
  3.805151618741797
  6.712655701212945
  7.998264854429905
  6.060583898072405
  5.521087185487601
  2.7055869235950736
  5.0290077633451125
  5.453657555385584
  4.151577904662954
  ⋮
  6.069863195453504
  3.5393345168491726
  6.63972925363036
  1.557584451949321
  6.638641029683351
  4.7355038874844375
  1.9879226229409959
  2.1208535392806
  9.638320384901153
  5.181010313213452
 -0.4511953354367222
  8.205692469575357

In [12]:
@which  sample(model, spl, 100000; param_names=["μ", "σ"])

(::StatsBase.var"#sample##kw")(::Any, ::typeof(sample), model::AbstractMCMC.AbstractModel, sampler::AbstractMCMC.AbstractSampler, arg) in AbstractMCMC at C:\Users\f50017812\.julia\packages\AbstractMCMC\oou1a\src\sample.jl:15

In [13]:
pkgs = Pkg.installed();
pkgs["Datetime"]

LoadError: [91mUndefVarError: Pkg not defined[39m

In [1]:
using Turing

In [2]:
@model function gdemo(x, y)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    x ~ Normal(m, sqrt(s))
    y ~ Normal(m, sqrt(s))
end

mod = gdemo(1.5, 2)
alg = IS()
n_samples = 1000

chn = sample(mod, alg, n_samples)

Chains MCMC chain (1000×3×1 Array{Float64,3}):

Log evidence      = -3.713594836897286
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = m, s
internals         = lp

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

           m    0.0801    1.8935     0.0599    0.0783   965.4217    1.0009
           s    2.9610    3.8789     0.1227    0.1100   998.3367    0.9994

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           m   -3.3181   -0.8196    0.0697    0.9980    4.0710
           s    0.5105    1.1178    1.8518    3.2459   12.7656


In [3]:
VERSION

v"1.5.3"

In [5]:
using Flux

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1278


In [10]:
f(x) = 3x^2 + 2x + 1
df(x) = gradient(f, x)[1];

In [11]:
df(2)

14

In [13]:
W = rand(2, 5)
b = rand(2)

2-element Array{Float64,1}:
 0.3604442874812279
 0.1661717171484125

In [6]:
x = 10
function change!(y)
    y = 17
end

change!(x)

17

In [12]:
module MyModule

export x, y

x() = "x"
y() = "y"
p() = "p"

end


Main.MyModule

In [18]:
fs(x) = x*2

fs (generic function with 1 method)

In [32]:
#fs isa Function
asd =3
asd isa Int

true

In [34]:
DensityModel(fs)

DensityModel{typeof(fs)}(fs)

In [35]:
# Define the components of a basic model.
insupport(θ) = θ[2] >= 0
dist(θ) = Normal(θ[1], θ[2])
density(θ) = insupport(θ) ? sum(logpdf.(dist(θ), data)) : -Inf

# Construct a DensityModel.
model = DensityModel(density)

DensityModel{typeof(density)}(density)

In [37]:
model.π

LoadError: [91mtype DensityModel has no field π[39m

In [40]:
fieldnames(typeof(model))

(:ℓπ,)

In [41]:
model:ℓπ

LoadError: [91mUndefVarError: ℓπ not defined[39m

In [48]:
das(s) = s + 2
das(3)

5

In [159]:
for _ in 1:0
    print(2)
end